#사용법

1. 우측 상단 '로그인'

2. 좌측 상단 '실습 모드에서 열기' 또는 'Drive에 사본 저장'

※ 각각의 셀은 셀 좌측 상단 실행 버튼을 통해 실행할 수 있습니다.

※ 혹은 CTRL + ENTER로 실행 가능합니다.

※ 실행 중 '경고: 이 노트는 Google에서 작성하지 않았습니다.'라는 창이 뜰 경우, '실행 전에 모든 런타임 재설정'란에 체크 후 '무시하고 계속하기'를 하시면 됩니다.

본 실습 참고 링크: https://medium.com/@hugmanskj/hands-on-bpe-byte-pair-encoding-%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80-%EA%B5%AC%ED%98%84-6bfef6f80f3b

# LLM과 Tokenizer 사용해보기

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# 모델과 토크나이저 로드(둘은 한몸!)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

thinking content: <think>
Okay, the user wants a short introduction to a large language model. Let me start by recalling what I know about them. Large language models, like BERT or GPT, are big neural networks designed to understand and generate human language. They can process text, answer questions, and even write creative content.

I should mention their main capabilities: understanding and generating text, answering questions, and adapting to different contexts. It's important to highlight their effectiveness in various tasks. Maybe include examples like BERT's ability to understand complex sentences or GPT's response to diverse queries.

Wait, the user might be a student or someone learning about AI, so clarity is key. I need to keep it concise but informative. Also, make sure to mention their use cases without getting too technical. Avoid jargon unless necessary. Let me structure it: start with the general definition, then the key features, and then examples. Check for any missin

In [57]:
# 질문 정의
prompt = "독도는 어느나라 땅이야?"

# 입력을 미리 정의된 채팅 Template 형식으로 바꾸는 작업
# LLM이 채팅이 가능해지면서 필요해진 작업입니다.
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)

print(text.strip())

<|im_start|>user
독도는 어느나라 땅이야?<|im_end|>
<|im_start|>assistant
<think>

</think>


In [62]:
# 입력을 Tokenization 하는 작업
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [65]:
model_inputs

{'input_ids': tensor([[151644,    872,    198, 129502,  47985,  16560, 133487,  60315,  50340,
           5140,    243,    227,  12802,  89659,     30, 151645,    198, 151644,
          77091,    198, 151667,    271, 151668,    271]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [83]:
for id in model_inputs['input_ids'][0]:
   print(int(id), f"'{tokenizer.decode(id)}'")

151644 '<|im_start|>'
872 'user'
198 '
'
129502 '독'
47985 '도'
16560 '는'
133487 ' 어느'
60315 '나'
50340 '라'
5140 ' �'
243 '�'
227 '�'
12802 '이'
89659 '야'
30 '?'
151645 '<|im_end|>'
198 '
'
151644 '<|im_start|>'
77091 'assistant'
198 '
'
151667 '<think>'
271 '

'
151668 '</think>'
271 '

'


In [82]:
print(tokenizer(" 땅"))
print(tokenizer("땅"))

{'input_ids': [5140, 243, 227], 'attention_mask': [1, 1, 1]}
{'input_ids': [139118], 'attention_mask': [1]}


In [78]:
tokenizer.decode([5140, 243, 227])

' 땅'

In [55]:
# 모델이 답변을 생성
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1000
)

In [56]:
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() # output에 input도 포함되어 있기 때문에 input은 제외하는 작업
print(output_ids) # 답변도 토큰 Id로 출력됨

[129502, 47985, 16560, 3070, 56039, 124785, 334, 12802, 5140, 243, 227, 12802, 89659, 13, 151645]


In [50]:
# 답변 토큰 ID를 다시 인간이 이해할 수 있는 단어로 변경
content = tokenizer.decode(output_ids, skip_special_tokens=False).strip("\n")

print("답변:", content)

답변: 독도는 **미국**이 주로 땅으로 사용하고 있으며, 독도는 **미국의 자치영사**로 볼 수 있습니다. 독도는 미국의 도시로 볼 수 있습니다.<|im_end|>


# 학습이 완료된 Tokenizer 사용하기

In [84]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 토크나이저 비쥬얼라이제이션을 위한 작업
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]


def show_tokens(sentence, tokenizer):
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [85]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # WordPiece 기반(자모 단위로 자를 수 있으면 잘라서 처리)
gpt_tokenizer = AutoTokenizer.from_pretrained("Xenova/gpt-4") # BPE 기반

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

In [108]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
오징어 게임
कि
"""

In [109]:
show_tokens(text, bert_tokenizer)

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 ᄋ ##ᅩ ##ᄌ ##ᅵ ##ᆼ ##ᄋ ##ᅥ ᄀ ##ᅦ ##ᄋ ##ᅵ ##ᆷ क ##ि [SEP] 

In [110]:
show_tokens(text, gpt_tokenizer)


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 오 � � 어  게 임 
 क ि 
 

In [95]:
# 두 Tokenizer의 차이가 뭘까요?

In [96]:
# 한글에 특화된 토크나이저 테스트
kanana_tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-1.5-v-3b-instruct")

In [111]:
show_tokens(text, kanana_tokenizer)

<|begin_of_text|> 
 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 오 징 어  게임 
 क ि 
 

# 토크나이저 직접 학습하기

## 학습 데이터(corpus) 다운로드

tokenizer 실습을 위한 데이터를 다운로드 합니다.

wikipedia-api를 통해 원하는 위키피디아 페이지를 다운로드 합니다.

In [ ]:
!pip install wikipedia-api

In [ ]:
# 원하는 주제를 선정하세요(위키피디아에 있는 주제)
# 문서 길이가 너무 짧은(10000자 이하) 문서는 지양
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia(user_agent='test-agent', language='ko')
wiki_data = wiki_wiki.page('박지성')
print("문서 길이: " + str(len(wiki_data.text)))
print(wiki_data.text)

제목: 박지성
요약: 박지성(1981년 2월 25일~)은 대한민국의 은퇴한 프로 축구 선수로 현역 시절 포지션은 윙어, 미드필더이다. 현재 전북 현대 모터스의 클럽 고문으로 재직 중이다. 서울에서 태어난 그는 선수로 활동하는 동안 트로피 19개를 획득했다. UEFA 챔피언스리그에서 우승한 최초의 아시아 축구 선수이고 챔피언스리그 결승전에 진출한 2명의 아시아 축구 선수(박지성, 손흥민) 중 한 명이자 FIFA 클럽 월드컵에서 우승을 한 최초의 아시아인 선수이다. 그는 뛰어난 체력과 훈련, 프로 의식으로 유명했으며, 그의 지구력으로 인해 "폐 3개 가진 박(Three-Lungs Park)"이라는 별명을 얻었다.
어린 시절부터 축구를 시작한 그는 명지대학교 축구부에서 활동했으며, 2000년에 일본으로 건너가 교토 퍼플 상가에서 활동하며 프로 선수 경력을 시작했다. 그 후 2003년에 대한민국 국가대표팀 감독을 맡았던 거스 히딩크가 네덜란드로 돌아와 감독을 맡은 팀인 네덜란드의 PSV 에인트호번에 입단하여 유럽 리그로 진출을 했다. PSV가 2004-05년 UEFA 챔피언스리그 준결승에 진출한 후 맨체스터 유나이티드 FC의 감독 알렉스 퍼거슨에게 인정을 받아 2005년 7월에 맨체스터 유나이티드와 계약을 맺었다. 그는 프리미어리그에서 4번 우승했으며, 2007-08년 UEFA 챔피언스리그, 2008년 FIFA 클럽 월드컵에서 우승하는 데에 기여하였다. 이후 주전 출전 횟수가 감소하자 2012년 7월 퀸스 파크 레인저스 FC로 이적했다. 그러나 이적 시즌에 자신의 부상과 소속팀의 강등으로 인해 2013-14 시즌에 임대 형식으로 PSV 에인트호번에 합류했다. 이후 2014년에 PSV에서의 활동을 마지막으로 현역에서 은퇴했다.
대한민국 국가대표팀의 일원으로도 활동하여 A매치 100경기에 출전하여 13골을 넣었다. 그는 2002년 FIFA 월드컵에서 4위를 한 대한민국팀의 일원이었으며, 2006년 FIFA 월드컵과 2010년 FIFA 월드컵에서도 대한민국 국가대표로 참가

## Dictionary 생성

corpus에서 공백 기준으로 단어를 분리하고, 단어별 등장 횟수를 셉니다.


In [ ]:
# Calculate word frequency
from collections import defaultdict
from tokenizers import pre_tokenizers
word_freqs = defaultdict(int)
words_with_offsets = pre_tokenizers.WhitespaceSplit().pre_tokenize_str(wiki_data.text)
new_words = [word for word, offset in words_with_offsets]
for word in new_words:
    word_freqs[word] += 1

for word, freq in word_freqs.items():
    print(f"{word:>12}: {freq}") # Align numbers to the right

   박지성(1981년: 1
          2월: 6
      25일~)은: 1
       대한민국의: 4
         은퇴한: 1
          프로: 5
          축구: 10
         선수로: 5
          현역: 2
          시절: 3
        포지션은: 3
         윙어,: 1
     미드필더이다.: 1
          현재: 1
          전북: 5
          현대: 5
        모터스의: 2
          클럽: 6
        고문으로: 3
          재직: 1
        중이다.: 1
        서울에서: 1
         태어난: 1
          그는: 10
        활동하는: 1
          동안: 4
         트로피: 1
        19개를: 1
       획득했다.: 1
        UEFA: 20
    챔피언스리그에서: 1
         우승한: 1
         최초의: 4
         아시아: 10
        선수이고: 1
      챔피언스리그: 19
        결승전에: 5
         진출한: 3
         2명의: 1
     선수(박지성,: 1
        손흥민): 1
           중: 9
           한: 7
         명이자: 1
        FIFA: 29
       월드컵에서: 9
         우승을: 4
        아시아인: 3
       선수이다.: 2
         뛰어난: 3
         체력과: 1
         훈련,: 1
        의식으로: 1
      유명했으며,: 1
          그의: 5
       지구력으로: 1
          인해: 4
          "폐: 1
          3개: 2
          가진: 1
박(Three-Lungs: 1
   Park)"이라는: 1
 

In [ ]:
# Corpus에 등장하는 모든 character를 기본 vocabulary로 저장
alphabet = []
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("Base vocabulary:", alphabet)

Base vocabulary: ['!', '"', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', '~', '·', 'É', '–', '《', '》', '三', '兩', '朴', '雙', '龍', '가', '각', '간', '갈', '감', '갑', '갔', '강', '갖', '같', '개', '거', '건', '걷', '걸', '검', '것', '게', '겠', '겨', '격', '겪', '견', '결', '경', '계', '고', '골', '공', '과', '관', '광', '괴', '교', '구', '국', '군', '궈', '권', '궤', '귀', '그', '극', '근', '글', '금', '급', '기', '긱', '긴', '길', '김', '까', '깨', '깼', '께', '꼽', '꽂', '꾸', '꾼', '꿈', '꿔', '꿨', '뀌', '끄', '끌', '끔', '끝', '나', '난', '날', '남', '납', '났', '내', '낸', '냅', '냈', '너', '널', '넓', '넘', '넣', '네', '넬', '년', '노', '녹', '논', '농', '놓', '누', '눈', '뉴', '는', '능', '늦', '니', '닌', '닝', '다', '단', '닫', '달', '담', '당', '대', '더', '덕', '던', '덜', '데', '덴', '델', '도

In [ ]:
# 모든 단어를 character 단위로 분리
splits = {word: [c for c in word] for word in word_freqs.keys()}
print("Initial word splits:")
for word, split in splits.items():
    print(f"{word:>12}: {split}")

Initial word splits:
   박지성(1981년: ['박', '지', '성', '(', '1', '9', '8', '1', '년']
          2월: ['2', '월']
      25일~)은: ['2', '5', '일', '~', ')', '은']
       대한민국의: ['대', '한', '민', '국', '의']
         은퇴한: ['은', '퇴', '한']
          프로: ['프', '로']
          축구: ['축', '구']
         선수로: ['선', '수', '로']
          현역: ['현', '역']
          시절: ['시', '절']
        포지션은: ['포', '지', '션', '은']
         윙어,: ['윙', '어', ',']
     미드필더이다.: ['미', '드', '필', '더', '이', '다', '.']
          현재: ['현', '재']
          전북: ['전', '북']
          현대: ['현', '대']
        모터스의: ['모', '터', '스', '의']
          클럽: ['클', '럽']
        고문으로: ['고', '문', '으', '로']
          재직: ['재', '직']
        중이다.: ['중', '이', '다', '.']
        서울에서: ['서', '울', '에', '서']
         태어난: ['태', '어', '난']
          그는: ['그', '는']
        활동하는: ['활', '동', '하', '는']
          동안: ['동', '안']
         트로피: ['트', '로', '피']
        19개를: ['1', '9', '개', '를']
       획득했다.: ['획', '득', '했', '다', '.']
        UEFA: ['U', 'E', 'F', 'A']
    챔피언스리그에서: 

In [ ]:
# 단어 내 연속된 문자 쌍의 빈도를 계산하는 함수
# 각 단어내에서의 문자들의 조합이 얼마나 많이 나타나는지 계산
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

## vocabulary 생성 시작

In [ ]:
vocab_size = 5000  # 최종 vocabulary 크기 설정
vocab = ["<|endoftext|>"] + alphabet # 기본 vocab, 특수 토큰 추가
merges = {}  # 병합 규칙을 저장할 딕셔너리

# vocabulary 크기가 원하는 vocab_size보다 작을 동안 반복
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)  # 단어 내 문자 쌍의 빈도 계산
    if not pair_freqs:  # 더 이상 병합할 쌍이 없으면 종료
        break

    best_pair = max(pair_freqs, key=pair_freqs.get)  # 가장 빈도가 높은 문자 쌍 선택
    print("Best Pair : ", best_pair)
    new_token = ''.join(best_pair)  # 선택된 문자 쌍을 병합하여 새로운 토큰 생성
    vocab.append(new_token)  # vocabulary에 새로운 토큰 추가
    merges[best_pair] = new_token  # 병합 규칙 저장

    # 단어 분리 결과 업데이트
    for word in word_freqs:  # 모든 단어에 대해 반복
        split = splits[word]  # 단어의 현재 분리 결과 가져오기
        i = 0
        while i < len(split) - 1:  # 단어 내 모든 문자 쌍에 대해 반복
            if (split[i], split[i+1]) == best_pair:  # 현재 쌍이 best_pair와 일치하면
                split = split[:i] + [new_token] + split[i+2:]  # best_pair를 new_token으로 병합
            else:
                i += 1  # 다음 문자 쌍으로 이동
        splits[word] = split  # 업데이트된 분리 결과 저장

Best Pair :  ('2', '0')
Best Pair :  ('다', '.')
Best Pair :  ('에', '서')
Best Pair :  ('20', '0')
Best Pair :  ('20', '1')
Best Pair :  ('지', '성')
Best Pair :  ('박', '지성')
Best Pair :  ('경', '기')
Best Pair :  ('리', '그')
Best Pair :  ('으', '로')
Best Pair :  ('하', '였')
Best Pair :  ('했', '다.')
Best Pair :  ('F', 'A')
Best Pair :  ('시', '즌')
Best Pair :  ('하였', '다.')
Best Pair :  ('F', 'C')
Best Pair :  ('골', '을')
Best Pair :  ('경기', '에서')
Best Pair :  ('월', '드')
Best Pair :  ('월드', '컵')
Best Pair :  ('박지성', '은')
Best Pair :  ('기', '록')
Best Pair :  ('-', '0')
Best Pair :  ('와', '의')
Best Pair :  ('나', '이')
Best Pair :  ('축', '구')
Best Pair :  ('2', '년')
Best Pair :  ('선', '수')
Best Pair :  ('었', '다.')
Best Pair :  ('유', '나이')
Best Pair :  ('유나이', '티')
Best Pair :  ('유나이티', '드')
Best Pair :  ('고', ',')
Best Pair :  ('아', '시')
Best Pair :  ('맨', '체')
Best Pair :  ('맨체', '스')
Best Pair :  ('맨체스', '터')
Best Pair :  ('201', '0')
Best Pair :  ('우', '승')
Best Pair :  ('F', 'I')
Best Pair :  ('FI

In [ ]:
print("Final vocabulary:", vocab)
print("Merge rules:", merges)

Final vocabulary: ['<|endoftext|>', '!', '"', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', '~', '·', 'É', '–', '《', '》', '三', '兩', '朴', '雙', '龍', '가', '각', '간', '갈', '감', '갑', '갔', '강', '갖', '같', '개', '거', '건', '걷', '걸', '검', '것', '게', '겠', '겨', '격', '겪', '견', '결', '경', '계', '고', '골', '공', '과', '관', '광', '괴', '교', '구', '국', '군', '궈', '권', '궤', '귀', '그', '극', '근', '글', '금', '급', '기', '긱', '긴', '길', '김', '까', '깨', '깼', '께', '꼽', '꽂', '꾸', '꾼', '꿈', '꿔', '꿨', '뀌', '끄', '끌', '끔', '끝', '나', '난', '날', '남', '납', '났', '내', '낸', '냅', '냈', '너', '널', '넓', '넘', '넣', '네', '넬', '년', '노', '녹', '논', '농', '놓', '누', '눈', '뉴', '는', '능', '늦', '니', '닌', '닝', '다', '단', '닫', '달', '담', '당', '대', '더', '덕', '던', '덜',

In [ ]:
print("Number of vocabulary", len(vocab))
print("Final vocabulary:", vocab)
print("Merge rules:")
for (a, b), merge in merges.items():
    print(f"('{a}', '{b}') -> '{merge}'")

Number of vocabulary 5000
Final vocabulary: ['<|endoftext|>', '!', '"', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y', '~', '·', 'É', '–', '《', '》', '三', '兩', '朴', '雙', '龍', '가', '각', '간', '갈', '감', '갑', '갔', '강', '갖', '같', '개', '거', '건', '걷', '걸', '검', '것', '게', '겠', '겨', '격', '겪', '견', '결', '경', '계', '고', '골', '공', '과', '관', '광', '괴', '교', '구', '국', '군', '궈', '권', '궤', '귀', '그', '극', '근', '글', '금', '급', '기', '긱', '긴', '길', '김', '까', '깨', '깼', '께', '꼽', '꽂', '꾸', '꾼', '꿈', '꿔', '꿨', '뀌', '끄', '끌', '끔', '끝', '나', '난', '날', '남', '납', '났', '내', '낸', '냅', '냈', '너', '널', '넓', '넘', '넣', '네', '넬', '년', '노', '녹', '논', '농', '놓', '누', '눈', '뉴', '는', '능', '늦', '니', '닌', '닝', '다', '단', '닫', '달', '담', '당'

In [ ]:
# 실제 텍스트가 들어오면 학습 된 룰과 vocab에 따라서 텍스트를 분절
def tokenize(text):
    # 1. WhitespaceSplit을 사용하여 공백 기준으로 텍스트를 pre-tokenize합니다.
    pre_tokenize_result = pre_tokenizers.WhitespaceSplit().pre_tokenize_str(text)

    # 2. pre-tokenize 결과에서 단어만 추출합니다.
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]

    # 3. 각 단어를 문자 단위로 분리하여 splits 리스트에 저장합니다.
    # 예 “low” -> [‘l’, ‘o’, ‘w’]
    splits = [[l for l in word] for word in pre_tokenized_text]

    # 4. merges 딕셔너리에 저장된 병합 규칙을 적용하여 토큰을 병합합니다.
    for pair, merge in merges.items():  # 각 병합 규칙에 대해 반복
        for idx, split in enumerate(splits):  # 각 단어의 분리 결과에 대해 반복
            i = 0
            while i < len(split) - 1:  # 단어 내 모든 문자 쌍에 대해 반복
                if split[i] == pair[0] and split[i + 1] == pair[1]:  # 현재 쌍이 병합 규칙과 일치하면
                    split = split[:i] + [merge] + split[i + 2 :]  # 병합 규칙을 적용하여 병합
                else:
                    i += 1  # 다음 문자 쌍으로 이동
            splits[idx] = split  # 업데이트된 분리 결과 저장

    # 5. 모든 분리 결과를 하나의 리스트로 합쳐서 반환합니다.
    return sum(splits, [])

## 테스트

띄어쓰기가 되지 않은 텍스트로 분절을 해보고, 자주 등장하는 단어로 잘 분리가 되는지 확인합니다.

In [ ]:
# 분절된 토큰을 다시 원문으로 변환
def decode(tokens):
    text = ''
    for token in tokens:
        text += token +" "
    return text.strip()  # Remove leading/trailing whitespace

In [ ]:
test_sentence = "박지성선수는축구를잘합니다."
tokens = tokenize(test_sentence)
print(f"Tokenization result for '{test_sentence}':")
print(tokens)

Tokenization result for '박지성선수는축구를잘합니다.':
['박지성', '선수는', '축구를', '잘', '합니', '다.']


In [ ]:
# 분절된 토큰을 원문으로 변환
decoded_text = decode(tokens)
print(f"\nDecoded text: '{decoded_text}'")


Decoded text: '박지성 선수는 축구를 잘 합니 다.'
